In [ ]:
# Import modules
import numpy as np
import pandas as pd
import nltk


# Read in customer reviews file
from google.colab import files
uploaded = files.upload()

customer_reviews = pd.read_csv("customer reviews.csv")

print(len(customer_reviews))
# Display the data
customer_reviews

In [ ]:


# Inspect the new DataFrame
customer_reviews.head()

In [ ]:
nltk.download("punkt")
import re

# Import the SnowballStemmer to perform stemming
from nltk.stem.snowball import SnowballStemmer

# Create an English language SnowballStemmer object
stemmer = SnowballStemmer("english")

# Define a function to perform both stemming and tokenization
def tokenize_and_stem(text):

    # Tokenize by sentence, then by word
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

    # Filter out raw tokens to remove noise
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]

    # Stem the filtered_tokens
    stems = [stemmer.stem(t) for t in filtered_tokens]

    return stems

words_stemmed = tokenize_and_stem(" This book is ok. Not as good as I was expecting. Does address fear and is a lesson. But reminds me of an action movie that goes on to long.")
print(words_stemmed)

In [ ]:
# Import TfidfVectorizer to create TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer object with stopwords and tokenizer
# parameters for efficient processing of text
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,3))

In [ ]:
# Fit and transform the tfidf_vectorizer with the "review description" of each movie
# to create a vector representation of the review description
tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in customer_reviews["review description"]])

print(tfidf_matrix.shape)

In [ ]:
# Import k-means to perform clustering
from sklearn.cluster import KMeans

# Create a KMeans object with 5 clusters and save as km
km = KMeans(n_clusters=5)

# Fit the k-means object with tfidf_matrix
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

# Create a column cluster to denote the generated cluster for each books
customer_reviews["cluster"] = clusters

# Display number of films per cluster (clusters from 0 to 4)
customer_reviews['cluster'].value_counts()


In [ ]:
# Import cosine_similarity to calculate similarity of customer reviewa
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the similarity distance
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
# Import matplotlib.pyplot for plotting graphs
import matplotlib.pyplot as plt

# Configure matplotlib to display the output inline
%matplotlib inline

# Import modules necessary to plot dendrogram
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
# Create mergings matrix
mergings = linkage(similarity_distance, method='complete')

# Plot the dendrogram, using title as label column
dendrogram_ = dendrogram(mergings,
               labels=[x for x in customer_reviews["book name"]],
               leaf_rotation=90,
               leaf_font_size=16,
)

# Adjust the plot
fig = plt.gcf()
_ = [lbl.set_color('r') for lbl in plt.gca().get_xmajorticklabels()]
fig.set_size_inches(400, 80)

# Show the plotted dendrogram
plt.show()